In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/best_chin_final.pth
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000304.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000050.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000280.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000286.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000182.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000111.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000038.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000292.jpg
/kaggle/input/equine-exp3-minimal/equine_exp3_minimal/cropped_regions_50FPS/S8_Video/chin/000327.jpg
/kaggle/input/equ

### EXPERIMENT PIPELINE

In [2]:
"""
================================================================================
IMPROVED I3D + BI-LSTM: END-TO-END JOINT TRAINING
================================================================================
Key Improvements:
1. Fine-tune I3D backbone while training LSTM (not frozen)
2. Extract per-frame features → LSTM gets proper temporal sequence
3. Temporal attention mechanism for better aggregation
4. Stronger data augmentation
5. Gradient accumulation for stable training with small batches
================================================================================
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score
from torchvision import transforms
from torchvision.models.video import r3d_18
import random

# ==================== CONFIG ====================
class Config:
    BASE_DIR = Path("/kaggle/input/equine-exp3-minimal/equine_exp3_minimal")
    RIFE_CROPS_DIR = BASE_DIR / "cropped_regions_50FPS"
    
    # Training
    BATCH_SIZE = 2  # Small due to memory
    ACCUMULATION_STEPS = 4  # Effective batch = 8
    NUM_EPOCHS = 40
    LEARNING_RATE = 5e-5  # Lower for fine-tuning
    WEIGHT_DECAY = 1e-4
    PATIENCE = 12
    
    # Architecture
    CLIP_LENGTH = 16
    STRIDE_TRAIN = 8   # Overlapping clips
    STRIDE_VAL = 16
    LSTM_HIDDEN = 256
    LSTM_LAYERS = 2
    
    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Data
    CLASS_NAMES = ['Mild', 'Moderate']
    VIDEO_PAIN_LEVELS = {
        'S1_Video': 1, 'S2_Video': 0, 'S3_Video': 1, 'S4_Video': 1,
        'S5_Video': 1, 'S6_Video': 0, 'S7_Video': 1, 'S8_Video': 1,
        'S9_Video': 0, 'S10_Video': 1, 'S11_Video': 1, 'S12_Video': 0,
    }
    
    @classmethod
    def compute_class_weights(cls):
        counts = np.bincount([cls.VIDEO_PAIN_LEVELS[v] for v in cls.VIDEO_PAIN_LEVELS])
        total = counts.sum()
        return torch.tensor(total / (len(counts) * counts.astype(float)), dtype=torch.float32)

Config.CLASS_WEIGHTS = Config.compute_class_weights()

print("="*80)
print("IMPROVED I3D + BI-LSTM PIPELINE")
print("="*80)
print(f"Device: {Config.DEVICE}")
print(f"Effective Batch Size: {Config.BATCH_SIZE * Config.ACCUMULATION_STEPS}")
class_counts = np.bincount([Config.VIDEO_PAIN_LEVELS[v] for v in Config.VIDEO_PAIN_LEVELS])
for i, name in enumerate(Config.CLASS_NAMES):
    print(f"  {name}: {class_counts[i]} videos (weight: {Config.CLASS_WEIGHTS[i]:.3f})")
print("="*80)



IMPROVED I3D + BI-LSTM PIPELINE
Device: cuda
Effective Batch Size: 8
  Mild: 4 videos (weight: 1.500)
  Moderate: 8 videos (weight: 0.750)


In [3]:
# ==================== DATASET ====================
class ImprovedEquineDataset(Dataset):
    """
    Each sample = one 16-frame clip
    Much more training samples than video-level approach
    """
    def __init__(self, root_dir, df, region, mode='train'):
        self.root_dir = Path(root_dir)
        self.region = region
        self.mode = mode
        self.df = df
        self.clips = []
        self._build_index()
        
        # Augmentation
        if mode == 'train':
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomResizedCrop(112, scale=(0.7, 1.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3),
                transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(112),
                transforms.CenterCrop(112),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])
    
    def _build_index(self):
        stride = Config.STRIDE_TRAIN if self.mode == 'train' else Config.STRIDE_VAL
        for _, row in self.df.iterrows():
            video = row['video_name']
            label = row['pain_level']
            path = self.root_dir / video / self.region
            if not path.exists(): continue
            frames = sorted(path.glob("*.jpg"))
            if len(frames) < Config.CLIP_LENGTH: continue
            
            for i in range(0, len(frames) - Config.CLIP_LENGTH + 1, stride):
                self.clips.append({
                    'frames': frames[i:i + Config.CLIP_LENGTH],
                    'label': label,
                    'video': video
                })
        print(f"  {self.mode.upper()} {self.region}: {len(self.clips)} clips")
    
    def __len__(self):
        return len(self.clips)
    
    def __getitem__(self, idx):
        item = self.clips[idx]
        
        # Load frames
        frames = []
        for frame_path in item['frames']:
            img = cv2.imread(str(frame_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(img)
        
        # Apply spatial transforms frame-wise
        frames = [self.transform(f) for f in frames]   # list of [C,H,W]
        clip = torch.stack(frames)                     # [T, C, H, W]
        
        # Temporal augmentation (training only)
        if self.mode == 'train':
            # Simple and robust: reverse temporal order with some probability
            if random.random() < 0.3:
                clip = clip.flip(0)  # [T, C, H, W] reversed in time
        
        # Return [C, T, H, W] for I3D_BiLSTM
        return clip.permute(1, 0, 2, 3), item['label']




In [4]:
class FocalLoss(nn.Module):
    """
    Focal loss with optional class weights (alpha).
    alpha: tensor of shape [num_classes] or None
    gamma: focusing parameter (>0)
    """
    def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # inputs: [B, num_classes], targets: [B]
        ce = F.cross_entropy(inputs, targets, weight=self.alpha,
                             reduction="none")   # [B]
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce      # [B]

        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss


In [5]:
# ==================== MODEL ====================
class I3D_BiLSTM(nn.Module):
    """
    Improved I3D + Bi-LSTM with:
    - Fine-tunable I3D backbone
    - Per-frame feature extraction
    - Temporal attention
    - Residual connections
    """
    def __init__(self, num_classes=2, freeze_i3d=False):
        super().__init__()
        
        # I3D backbone (feature extractor)
        backbone = r3d_18(pretrained=True)
        # Remove final pooling and FC
        self.i3d_features = nn.Sequential(*list(backbone.children())[:-2])
        
        # Option to freeze I3D
        if freeze_i3d:
            for param in self.i3d_features.parameters():
                param.requires_grad = False
        
        # Adaptive pooling to get consistent spatial size
        self.adaptive_pool = nn.AdaptiveAvgPool3d((None, 1, 1))  # Keep T, pool H,W
        
        # Bi-LSTM for temporal modeling
        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=Config.LSTM_HIDDEN,
            num_layers=Config.LSTM_LAYERS,
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if Config.LSTM_LAYERS > 1 else 0
        )
        
        # Temporal attention
        self.attention = nn.Sequential(
            nn.Linear(Config.LSTM_HIDDEN * 2, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(Config.LSTM_HIDDEN * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        """
        Args:
            x: [B, C, T, H, W] = [B, 3, 16, 112, 112]
        Returns:
            logits: [B, num_classes]
        """
        B, C, T, H, W = x.shape
        
        # Extract I3D features
        feats = self.i3d_features(x)  # [B, 512, T', H', W']
        
        # Pool spatially, keep temporal
        feats = self.adaptive_pool(feats)  # [B, 512, T', 1, 1]
        feats = feats.squeeze(-1).squeeze(-1)  # [B, 512, T']
        feats = feats.permute(0, 2, 1)  # [B, T', 512]
        
        # Bi-LSTM
        lstm_out, _ = self.lstm(feats)  # [B, T', 512]
        
        # Temporal attention
        attn_scores = self.attention(lstm_out)  # [B, T', 1]
        attn_weights = F.softmax(attn_scores, dim=1)
        
        # Weighted sum
        context = (lstm_out * attn_weights).sum(dim=1)  # [B, 512]
        
        # Classify
        logits = self.classifier(context)
        
        return logits



In [6]:
# ==================== TRAINING ====================
def train_model(train_dataset, val_dataset, save_path):
    """
    Train I3D + Bi-LSTM with gradient accumulation and mixed precision
    """
    train_loader = DataLoader(
        train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True,
        num_workers=2, pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False,
        num_workers=2, pin_memory=True
    )
    
    # Model
    model = I3D_BiLSTM(num_classes=len(Config.CLASS_NAMES), freeze_i3d=False).to(Config.DEVICE)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nModel Parameters:")
    print(f"  Total: {total_params:,}")
    print(f"  Trainable: {trainable_params:,}")
    
    # Loss with class weights
    #criterion = nn.CrossEntropyLoss(weight=Config.CLASS_WEIGHTS.to(Config.DEVICE))
    criterion = FocalLoss(alpha=Config.CLASS_WEIGHTS.to(Config.DEVICE), gamma=2.0)

    # Optimizer - different LR for I3D vs LSTM
    i3d_params = list(model.i3d_features.parameters())
    lstm_params = list(model.lstm.parameters()) + list(model.attention.parameters()) + \
                  list(model.classifier.parameters())
    
    optimizer = torch.optim.AdamW([
        {'params': i3d_params, 'lr': Config.LEARNING_RATE * 0.1},  # Lower LR for I3D
        {'params': lstm_params, 'lr': Config.LEARNING_RATE}
    ], weight_decay=Config.WEIGHT_DECAY)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )
    
    # Mixed precision
    scaler = GradScaler()
    
    # Training state
    best_f1 = 0.0
    patience_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'val_f1': [], 'val_acc': []}
    
    print("\n" + "="*80)
    print("TRAINING I3D + BI-LSTM (JOINT END-TO-END)")
    print("="*80)
    
    for epoch in range(1, Config.NUM_EPOCHS + 1):
        # ==================== TRAINING ====================
        model.train()
        train_loss = 0.0
        optimizer.zero_grad()
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d}/{Config.NUM_EPOCHS}", ncols=100)
        for batch_idx, (clips, labels) in enumerate(pbar):
            clips = clips.to(Config.DEVICE)
            labels = labels.to(Config.DEVICE)
            
            # Mixed precision forward
            with autocast():
                outputs = model(clips)
                loss = criterion(outputs, labels)
                loss = loss / Config.ACCUMULATION_STEPS
            
            # Backward
            scaler.scale(loss).backward()
            
            # Gradient accumulation
            if (batch_idx + 1) % Config.ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            train_loss += loss.item() * Config.ACCUMULATION_STEPS
            pbar.set_postfix({'loss': f'{loss.item() * Config.ACCUMULATION_STEPS:.3f}'})
        
        avg_train_loss = train_loss / len(train_loader)
        
        # ==================== VALIDATION ====================
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for clips, labels in val_loader:
                clips = clips.to(Config.DEVICE)
                labels = labels.to(Config.DEVICE)
                
                outputs = model(clips)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                preds = outputs.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        val_f1 = f1_score(all_labels, all_preds, average='macro') if len(set(all_labels)) > 1 else 0.0
        val_acc = accuracy_score(all_labels, all_preds)
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_f1'].append(val_f1)
        history['val_acc'].append(val_acc)
        
        # Print summary
        print(f"\nEpoch {epoch:02d}:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}, F1: {val_f1:.4f}, Acc: {val_acc:.3f}")
        
        # LR scheduling
        scheduler.step(val_f1)
        
        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_f1': best_f1,
                'history': history
            }, save_path)
            print(f" Saved best model (F1={best_f1:.4f})")
        else:
            patience_counter += 1
            print(f" No improvement ({patience_counter}/{Config.PATIENCE})")
        
        # Early stopping
        if patience_counter >= Config.PATIENCE:
            print(f"\nEarly stopping at epoch {epoch}")
            break
    
    print(f"\nTraining complete! Best Val F1: {best_f1:.4f}")
    
    # Plot training curves
    plot_history(history, save_path.parent)
    
    return best_f1, history




In [7]:
def plot_history(history, save_dir):
    """Plot training curves"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # F1
    axes[1].plot(epochs, history['val_f1'], 'g-', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('F1 Score')
    axes[1].set_title('Validation F1')
    axes[1].grid(True, alpha=0.3)
    
    # Accuracy
    axes[2].plot(epochs, history['val_acc'], 'm-', linewidth=2)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Accuracy')
    axes[2].set_title('Validation Accuracy')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_dir / 'training_curves_i3d_bilstm.png', dpi=150, bbox_inches='tight')
    plt.close()



In [8]:

# ==================== TESTING ====================
def test_model(model_path, test_dataset):
    """Test on held-out set"""
    model = I3D_BiLSTM(num_classes=len(Config.CLASS_NAMES)).to(Config.DEVICE)
    checkpoint = torch.load(model_path, map_location=Config.DEVICE, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    all_preds = []
    all_labels = []
    
    print("\n" + "="*80)
    print("TESTING I3D + BI-LSTM")
    print("="*80)
    
    with torch.no_grad():
        for clips, labels in test_loader:
            clips = clips.to(Config.DEVICE)
            outputs = model(clips)
            preds = outputs.argmax(dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Metrics
    test_f1 = f1_score(all_labels, all_preds, average='macro')
    test_acc = accuracy_score(all_labels, all_preds)
    
    print(f"\nTest Accuracy: {test_acc:.3f}")
    print(f"Test Macro F1: {test_f1:.3f}")
    
    print("\n" + classification_report(all_labels, all_preds, 
                                       target_names=Config.CLASS_NAMES, digits=3))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=Config.CLASS_NAMES, yticklabels=Config.CLASS_NAMES)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('I3D + Bi-LSTM Test Confusion Matrix')
    plt.tight_layout()
    plt.savefig(Config.OUTPUT_DIR / 'confusion_matrix_i3d_bilstm.png', dpi=150)
    plt.close()
    
    return test_f1, test_acc

In [9]:
# ==================== MAIN ====================
def main():
    # Data split
    test_videos = ['S2_Video', 'S4_Video', 'S7_Video']
    train_videos = [v for v in Config.VIDEO_PAIN_LEVELS if v not in test_videos]
    val_videos = train_videos[-2:]
    train_v = train_videos[:-2]
    
    # Create dataframes
    train_df = pd.DataFrame([{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} 
                             for v in train_v])
    val_df = pd.DataFrame([{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} 
                           for v in val_videos])
    test_df = pd.DataFrame([{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} 
                            for v in test_videos])
    
    print("\nData Split:")
    print(f"  Train videos: {train_v}")
    print(f"  Val videos:   {val_videos}")
    print(f"  Test videos:  {test_videos}")

    
    # Create datasets
    train_dataset = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, train_df, 'chin', 'train')
    val_dataset = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, val_df, 'chin', 'val')
    test_dataset = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, test_df, 'chin', 'val')

    print("\nClip counts:")
    print(f"  Train clips: {len(train_dataset)}")
    print(f"  Val clips:   {len(val_dataset)}")
    print(f"  Test clips:  {len(test_dataset)}")
    # Optional safety check
    if len(train_dataset) == 0 or len(val_dataset) == 0:
        print("\nNo clips in train or val split; check Config.RIFE_CROPS_DIR and folder names.")
        return
    
    # Train
    save_path = Config.OUTPUT_DIR / 'best_i3d_bilstm.pth'
    best_f1, history = train_model(train_dataset, val_dataset, save_path)
    
    # Test
    test_f1, test_acc = test_model(save_path, test_dataset)
    
    # Save summary
    summary = {
        'best_val_f1': float(best_f1),
        'test_f1': float(test_f1),
        'test_acc': float(test_acc),
        'train_videos': train_v,
        'val_videos': val_videos,
        'test_videos': test_videos
    }
    
    import json
    with open(Config.OUTPUT_DIR / 'summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print("\n" + "="*80)
    print("EXPERIMENT COMPLETE!")
    print("="*80)
    print(f"Best Val F1: {best_f1:.4f}")
    print(f"Test F1: {test_f1:.4f}")
    print(f"Test Acc: {test_acc:.3f}")
    print(f"\nResults saved to: {Config.OUTPUT_DIR}")
    print("="*80)

### TRAIN WITH CROSS ENTROPY CRITERION

In [10]:
from pathlib import Path

# CE joint model (clip + video eval)
Config.OUTPUT_DIR = Path("/kaggle/working/i3d_bilstm_ce")
Config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [11]:
if __name__ == '__main__':
    main()


Data Split:
  Train videos: ['S1_Video', 'S3_Video', 'S5_Video', 'S6_Video', 'S8_Video', 'S9_Video', 'S10_Video']
  Val videos:   ['S11_Video', 'S12_Video']
  Test videos:  ['S2_Video', 'S4_Video', 'S7_Video']
  TRAIN chin: 169 clips
  VAL chin: 27 clips
  VAL chin: 46 clips

Clip counts:
  Train clips: 169
  Val clips:   27
  Test clips:  46


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:00<00:00, 204MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipy


Model Parameters:
  Total: 36,550,467
  Trainable: 36,550,467

TRAINING I3D + BI-LSTM (JOINT END-TO-END)


Epoch 01/40:   0%|                                                           | 0/84 [00:00<?, ?it/s]/tmp/ipykernel_20/3509960944.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 01/40: 100%|██████████████████████████████████████| 84/84 [00:11<00:00,  7.22it/s, loss=0.279]



Epoch 01:
  Train Loss: 0.1433
  Val Loss: 0.2714, F1: 0.3415, Acc: 0.519
 Saved best model (F1=0.3415)


...


Epoch 18:
  Train Loss: 0.0043
  Val Loss: 1.0204, F1: 0.4214, Acc: 0.556
 No improvement (11/12)


Epoch 19/40:   0%|                                                           | 0/84 [00:00<?, ?it/s]/tmp/ipykernel_20/3509960944.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 19/40: 100%|██████████████████████████████████████| 84/84 [00:07<00:00, 11.93it/s, loss=0.004]



Epoch 19:
  Train Loss: 0.0358
  Val Loss: 1.0707, F1: 0.4214, Acc: 0.556
 No improvement (12/12)

Early stopping at epoch 19

Training complete! Best Val F1: 0.4923


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



TESTING I3D + BI-LSTM

Test Accuracy: 0.478
Test Macro F1: 0.357

              precision    recall  f1-score   support

        Mild      0.042     0.500     0.077         2
    Moderate      0.955     0.477     0.636        44

    accuracy                          0.478        46
   macro avg      0.498     0.489     0.357        46
weighted avg      0.915     0.478     0.612        46


EXPERIMENT COMPLETE!
Best Val F1: 0.4923
Test F1: 0.3566
Test Acc: 0.478

Results saved to: /kaggle/working/i3d_bilstm_ce


### TRAIN WITH FOCAL LOSS CRITERION

In [12]:
Config.OUTPUT_DIR = Path("/kaggle/working/i3d_bilstm_focal")
Config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [13]:
if __name__ == '__main__':
    main()


Data Split:
  Train videos: ['S1_Video', 'S3_Video', 'S5_Video', 'S6_Video', 'S8_Video', 'S9_Video', 'S10_Video']
  Val videos:   ['S11_Video', 'S12_Video']
  Test videos:  ['S2_Video', 'S4_Video', 'S7_Video']
  TRAIN chin: 169 clips
  VAL chin: 27 clips
  VAL chin: 46 clips

Clip counts:
  Train clips: 169
  Val clips:   27
  Test clips:  46


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_20/3509960944.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Model Parameters:
  Total: 36,550,467
  Trainable: 36,550,467

TRAINING I3D + BI-LSTM (JOINT END-TO-END)


Epoch 01/40:   0%|                                                           | 0/84 [00:00<?, ?it/s]/tmp/ipykernel_20/3509960944.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 01/40: 100%|██████████████████████████████████████| 84/84 [00:07<00:00, 11.80it/s, loss=0.239]



Epoch 01:
  Train Loss: 0.1453
  Val Loss: 0.2440, F1: 0.3250, Acc: 0.481
 Saved best model (F1=0.3250)


...


Epoch 25:
  Train Loss: 0.0539
  Val Loss: 1.0813, F1: 0.3415, Acc: 0.519
 No improvement (11/12)


Epoch 26/40:   0%|                                                           | 0/84 [00:00<?, ?it/s]/tmp/ipykernel_20/3509960944.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 26/40: 100%|██████████████████████████████████████| 84/84 [00:09<00:00,  8.86it/s, loss=0.002]



Epoch 26:
  Train Loss: 0.0541
  Val Loss: 1.0546, F1: 0.3415, Acc: 0.519
 No improvement (12/12)

Early stopping at epoch 26

Training complete! Best Val F1: 0.4671


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



TESTING I3D + BI-LSTM

Test Accuracy: 0.065
Test Macro F1: 0.065

              precision    recall  f1-score   support

        Mild      0.044     1.000     0.085         2
    Moderate      1.000     0.023     0.044        44

    accuracy                          0.065        46
   macro avg      0.522     0.511     0.065        46
weighted avg      0.958     0.065     0.046        46


EXPERIMENT COMPLETE!
Best Val F1: 0.4671
Test F1: 0.0648
Test Acc: 0.065

Results saved to: /kaggle/working/i3d_bilstm_focal


### VIDEO LEVEL EVALUATION (CE Checkpoint)

In [14]:
import pandas as pd

# Same split you used for the joint model
test_videos = ['S2_Video', 'S4_Video', 'S7_Video']
train_videos = [v for v in Config.VIDEO_PAIN_LEVELS if v not in test_videos]
val_videos   = train_videos[-2:]
train_v      = train_videos[:-2]

print("Train videos:", train_v)
print("Val videos:  ", val_videos)
print("Test videos: ", test_videos)

# Create DataFrames
train_df = pd.DataFrame(
    [{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} for v in train_v]
)
val_df = pd.DataFrame(
    [{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} for v in val_videos]
)
test_df = pd.DataFrame(
    [{'video_name': v, 'pain_level': Config.VIDEO_PAIN_LEVELS[v]} for v in test_videos]
)

display(train_df)
display(val_df)
display(test_df)


Train videos: ['S1_Video', 'S3_Video', 'S5_Video', 'S6_Video', 'S8_Video', 'S9_Video', 'S10_Video']
Val videos:   ['S11_Video', 'S12_Video']
Test videos:  ['S2_Video', 'S4_Video', 'S7_Video']


,video_name,pain_level
0,S1_Video,1
1,S3_Video,1
2,S5_Video,1
3,S6_Video,0
4,S8_Video,1
5,S9_Video,0
6,S10_Video,1


,video_name,pain_level
0,S11_Video,1
1,S12_Video,0


,video_name,pain_level
0,S2_Video,0
1,S4_Video,1
2,S7_Video,1


In [15]:
from torch.utils.data import Dataset
from pathlib import Path
import cv2
import torch
from torchvision import transforms
import random

class ImprovedEquineDataset(Dataset):
    """
    Each sample = one 16-frame clip from a given video & region.
    mode in {'train','val'} controls stride and augmentation.
    """
    def __init__(self, root_dir, df, region, mode='train'):
        self.root_dir = Path(root_dir)
        self.region = region
        self.mode = mode
        self.df = df
        self.clips = []
        self.video_ids = []   # one entry per clip
        self._build_index()
        
        if mode == 'train':
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomResizedCrop(112, scale=(0.7, 1.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3),
                transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225]),
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(112),
                transforms.CenterCrop(112),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225]),
            ])
    
    def _build_index(self):
        stride = Config.STRIDE_TRAIN if self.mode == 'train' else Config.STRIDE_VAL
        for _, row in self.df.iterrows():
            video = row['video_name']
            label = row['pain_level']
            path = self.root_dir / video / self.region
            if not path.exists():
                continue
            frames = sorted(path.glob("*.jpg"))
            if len(frames) < Config.CLIP_LENGTH:
                continue
            
            for i in range(0, len(frames) - Config.CLIP_LENGTH + 1, stride):
                self.clips.append({
                    'frames': frames[i:i + Config.CLIP_LENGTH],
                    'label': label,
                    'video': video
                })
                self.video_ids.append(video)
        print(f"  {self.mode.upper()} {self.region}: {len(self.clips)} clips")
    
    def __len__(self):
        return len(self.clips)
    
    def __getitem__(self, idx):
        item = self.clips[idx]
        
        frames = []
        for frame_path in item['frames']:
            img = cv2.imread(str(frame_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(img)
        
        frames = [self.transform(f) for f in frames]   # list of [C,H,W]
        clip = torch.stack(frames)                     # [T,C,H,W]
        
        if self.mode == 'train':
            if random.random() < 0.3:
                clip = clip.flip(0)  # reverse time
        
        return clip.permute(1, 0, 2, 3), item['label']  # [C,T,H,W], label


In [16]:
#  Recreate datasets with this class
train_dataset = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, train_df, 'chin', 'train')
val_dataset   = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, val_df,   'chin', 'val')
test_dataset  = ImprovedEquineDataset(Config.RIFE_CROPS_DIR, test_df,  'chin', 'val')

print("Train clips:", len(train_dataset))
print("Val clips:  ", len(val_dataset))
print("Test clips: ", len(test_dataset))
print("First few test video_ids:", test_dataset.video_ids[:10])


  TRAIN chin: 169 clips
  VAL chin: 27 clips
  VAL chin: 46 clips
Train clips: 169
Val clips:   27
Test clips:  46
First few test video_ids: ['S2_Video', 'S2_Video', 'S4_Video', 'S4_Video', 'S4_Video', 'S4_Video', 'S4_Video', 'S4_Video', 'S4_Video', 'S4_Video']


In [17]:
# Video‑level evaluation helpers
import numpy as np
from sklearn.metrics import classification_report, f1_score, accuracy_score
from collections import defaultdict, Counter

def evaluate_video_level(model, test_loader, device, video_ids):
    model.eval()
    video_logits = defaultdict(list)
    video_labels = {}

    with torch.no_grad():
        idx_global = 0
        for clips, labels in test_loader:
            clips = clips.to(device)
            logits = model(clips)  # [B, num_classes]
            B = clips.size(0)
            for i in range(B):
                vid = video_ids[idx_global]
                video_logits[vid].append(logits[i].cpu())
                video_labels[vid] = labels[i].item()
                idx_global += 1

    video_predictions = {}
    for vid, logit_list in video_logits.items():
        all_logits = torch.stack(logit_list, dim=0)  # [num_clips, num_classes]
        avg_logits = all_logits.mean(dim=0)
        video_predictions[vid] = torch.argmax(avg_logits).item()

    return video_predictions, video_labels

def print_video_level_results(video_predictions, video_labels, class_names=['Mild','Moderate']):
    print("\n" + "="*80)
    print("VIDEO-LEVEL EVALUATION (Joint I3D + Bi-LSTM)")
    print("="*80)

    print("\nPer-Video Predictions:")
    print("-" * 50)
    for vid in sorted(video_predictions.keys()):
        true_label = class_names[video_labels[vid]]
        pred_label = class_names[video_predictions[vid]]
        correct = "✓" if video_labels[vid] == video_predictions[vid] else "✗"
        print(f"{vid:8s}  True: {true_label:10s}  Pred: {pred_label:10s}  {correct}")

    y_true = [video_labels[vid] for vid in sorted(video_predictions.keys())]
    y_pred = [video_predictions[vid] for vid in sorted(video_predictions.keys())]

    accuracy = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro')

    print("\n" + "-" * 50)
    print(f"Video-Level Accuracy: {accuracy:.3f} ({sum(np.array(y_true)==np.array(y_pred))}/{len(y_true)} videos)")
    print(f"Video-Level Macro F1: {macro_f1:.3f}")
    print("\nPer-Class Metrics:")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=3))
    print("="*80 + "\n")

    return accuracy, macro_f1

def test_joint_model_properly(model, test_dataset, batch_size=4, device='cuda'):
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    video_ids = test_dataset.video_ids

    print("\nTest Set Composition (clips per video):")
    vid_counts = Counter(video_ids)
    for vid, count in sorted(vid_counts.items()):
        print(f"   {vid}: {count} clips")

    video_preds, video_labels = evaluate_video_level(
        model, test_loader, device, video_ids
    )

    acc, macro_f1 = print_video_level_results(
        video_preds, video_labels, class_names=['Mild', 'Moderate']
    )

    return acc, macro_f1, video_preds, video_labels


In [18]:
# Load the trained joint model
# Rebuild the same joint model
joint_model = I3D_BiLSTM(num_classes=len(Config.CLASS_NAMES)).to(Config.DEVICE)

ckpt_path = Config.OUTPUT_DIR / 'best_i3d_bilstm.pth'
ckpt = torch.load(ckpt_path, map_location=Config.DEVICE, weights_only=False)
joint_model.load_state_dict(ckpt['model_state_dict'])
joint_model.eval()

print("Loaded joint model from:", ckpt_path)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded joint model from: /kaggle/working/i3d_bilstm_focal/best_i3d_bilstm.pth


In [19]:
video_acc, video_f1, video_preds, video_labels = test_joint_model_properly(
    joint_model,
    test_dataset,
    batch_size=4,
    device=Config.DEVICE
)

print("Final VIDEO-LEVEL metrics for joint I3D + Bi-LSTM (CE):")
print(f"  Accuracy: {video_acc:.3f}")
print(f"  Macro F1: {video_f1:.3f}")



Test Set Composition (clips per video):
   S2_Video: 2 clips
   S4_Video: 22 clips
   S7_Video: 22 clips

VIDEO-LEVEL EVALUATION (Joint I3D + Bi-LSTM)

Per-Video Predictions:
--------------------------------------------------
S2_Video  True: Mild        Pred: Mild        ✓
S4_Video  True: Moderate    Pred: Mild        ✗
S7_Video  True: Moderate    Pred: Mild        ✗

--------------------------------------------------
Video-Level Accuracy: 0.333 (1/3 videos)
Video-Level Macro F1: 0.250

Per-Class Metrics:
              precision    recall  f1-score   support

        Mild      0.333     1.000     0.500         1
    Moderate      0.000     0.000     0.000         2

    accuracy                          0.333         3
   macro avg      0.167     0.500     0.250         3
weighted avg      0.111     0.333     0.167         3


Final VIDEO-LEVEL metrics for joint I3D + Bi-LSTM (CE):
  Accuracy: 0.333
  Macro F1: 0.250


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# Show top-level contents of working dir
!ls -R /kaggle/working

/kaggle/working:
i3d_bilstm_ce  i3d_bilstm_focal  __notebook__.ipynb

/kaggle/working/i3d_bilstm_ce:
best_i3d_bilstm.pth		 summary.json
confusion_matrix_i3d_bilstm.png  training_curves_i3d_bilstm.png

/kaggle/working/i3d_bilstm_focal:
best_i3d_bilstm.pth		 summary.json
confusion_matrix_i3d_bilstm.png  training_curves_i3d_bilstm.png


In [21]:
%cd /kaggle/working

# Zip each folder separately
!zip -r i3d_bilstm_ce_outputs.zip i3d_bilstm_ce
!zip -r i3d_bilstm_focal_outputs.zip i3d_bilstm_focal
!zip -r i3d_bilstm_improved_outputs.zip i3d_bilstm_improved

# Check they exist
!ls -lh


/kaggle/working
  adding: i3d_bilstm_ce/ (stored 0%)
  adding: i3d_bilstm_ce/summary.json (deflated 60%)
  adding: i3d_bilstm_ce/best_i3d_bilstm.pth (deflated 8%)
  adding: i3d_bilstm_ce/training_curves_i3d_bilstm.png (deflated 8%)
  adding: i3d_bilstm_ce/confusion_matrix_i3d_bilstm.png (deflated 19%)
  adding: i3d_bilstm_focal/ (stored 0%)
  adding: i3d_bilstm_focal/summary.json (deflated 59%)
  adding: i3d_bilstm_focal/best_i3d_bilstm.pth (deflated 8%)
  adding: i3d_bilstm_focal/training_curves_i3d_bilstm.png (deflated 9%)
  adding: i3d_bilstm_focal/confusion_matrix_i3d_bilstm.png (deflated 19%)
	zip warning: name not matched: i3d_bilstm_improved

zip error: Nothing to do! (try: zip -r i3d_bilstm_improved_outputs.zip . -i i3d_bilstm_improved)
total 772M
drwxr-xr-x 2 root root 4.0K Dec  7 08:07 i3d_bilstm_ce
-rw-r--r-- 1 root root 385M Dec  7 08:11 i3d_bilstm_ce_outputs.zip
drwxr-xr-x 2 root root 4.0K Dec  7 08:11 i3d_bilstm_focal
-rw-r--r-- 1 root root 385M Dec  7 08:12 i3d_bilstm_fo